In [1]:
import numpy as np 
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import string

2023-06-06 13:15:48.728663: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [2]:
data = pd.read_csv('/home/codetrade/Downloads/CSV/tripadvisor_hotel_reviews.csv')
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [3]:
data.describe()

,Rating
count,20491.000000
mean,3.952223
std,1.233030
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [4]:
data.isnull().mean()

Review    0.0
Rating    0.0
dtype: float64

In [5]:
import preprocess_kgptalkie as ps
import re

In [6]:
data['Rating'].value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

In [7]:
def get_clean_data(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [8]:
data['Review']=data['Review'].apply(lambda x: get_clean_data(x))

/home/codetrade/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4 experience hotel monaco seatt...,3
3,unique great stay wonderful time hotel monaco ...,5
4,great stay great stay went seahawk game awesom...,5


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [11]:
tfid=TfidfVectorizer(max_features=20000,ngram_range=(1,3),analyzer='char')

In [12]:
X =tfid.fit_transform(data['Review'])
y = data['Rating']

In [13]:
X.shape,y.shape

((20491, 16355), (20491,))

In [14]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=0) 

In [15]:
clf=LinearSVC()
clf.fit(X_train,y_train)

LinearSVC()

In [16]:
y_pred=clf.predict(X_test)

In [17]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.62      0.68      0.65       285
           2       0.42      0.28      0.34       355
           3       0.46      0.15      0.23       471
           4       0.50      0.50      0.50      1203
           5       0.68      0.84      0.75      1785

    accuracy                           0.60      4099
   macro avg       0.54      0.49      0.49      4099
weighted avg       0.58      0.60      0.57      4099

